In [1]:
!pip install torch transformers sentencepiece sacrebleu bert_score rouge_score datasets tensorflow-text sentence_splitter

^C


     ---------------------------------------- 0.0/45.0 kB ? eta -:--:--
     ---------------------------------------- 45.0/45.0 kB 1.1 MB/s eta 0:00:00


#### ThanhJamieAI/FPT_Paraphrase
#### ThanhJamieAI/Thanh_ParaphraseV3
#### ThanhJamieAI/ParapharseV8_8E_4B

In [1]:
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
import sentencepiece
import torch
from sentence_splitter import SentenceSplitter
import evaluate

model_name = "ThanhJamieAI/ParapharseV8_8E_4B"
device = "cuda" if torch.cuda.is_available() else "cpu"
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name).to(device)
bleu = evaluate.load('bleu') # 'bleu': 1.0, 'precisions': [1.0, 1.0, 1.0, 1.0], 'brevity_penalty': 1.0, 'length_ratio': 1.1666666666666667, 'translation_length': 7, 'reference_length': 6}
ter = evaluate.load('ter') # {'score': ter_score, 'num_edits': num_edits, 'ref_length': ref_length}
rouge = evaluate.load('rouge') # ['rouge1', 'rouge2', 'rougeL', 'rougeLsum']
meteor = evaluate.load('meteor')  # Its values range from 0 to 1
sacrebleu = evaluate.load('sacrebleu') # score: BLEU score | counts: Counts | totals: Totals | precisions: Precisions | bp: Brevity penalty | sys_len: predictions length | ref_len: reference length



[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [3]:
# Ví dụ sử dụng:
splitter = SentenceSplitter(language='en')
text = "Where do they get a random paragraph? he wondered as he clicked the generate button. Do they just write a random paragraph or do they get it somewhere? At that moment he read the random paragraph and realized it was about random paragraphs and his world would never be the same."
sentence_list = splitter.split(text)

batch = tokenizer(sentence_list,truncation=True, padding='longest', max_length=100, return_tensors="pt").to(device)
translated = model.generate(**batch, max_length=1024, num_beams=10, num_return_sequences=2, temperature=1.5, do_sample = True)
tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
paraphrased_text = " ".join(tgt_text)
print(paraphrased_text)


Where do they get a random paragraph? Where do they get a random paragraph? Do you think they've written a random paragraph? Do they just write a random paragraph? At this moment, he read the random paragraph and realized it was about random paragraphs and his world would never be the same. At that moment, he looked back at the random paragraph and realized it was about random paragraphs and his world would never be the same.


In [3]:
splitter = SentenceSplitter(language='en')
text = "She didn't understand how changed worked. When she looked at today compared to yesterday, there was nothing that she could see that was different. Yet, when she looked at today compared to last year, she couldn't see how anything was ever the same."
sentence_list = splitter.split(text)
sentence_list

["She didn't understand how changed worked.",
 'When she looked at today compared to yesterday, there was nothing that she could see that was different.',
 "Yet, when she looked at today compared to last year, she couldn't see how anything was ever the same."]

In [11]:
len(sentence_list)

3

In [4]:
batch = tokenizer(sentence_list,truncation=True, padding='longest', max_length=100, return_tensors="pt").to(device)
translated = model.generate(**batch, max_length=1024, num_beams=3, num_return_sequences=3, temperature=1.5, do_sample = True)
tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)


In [4]:


def demo(sentence_list, num_return_sequences, num_beams,temperature,do_sample, input_string):
    dict_in_dict  = {}
    max_sentence_output = int(num_return_sequences * len(sentence_list))
    batch = tokenizer(sentence_list,truncation=True, padding='longest', max_length=100, return_tensors="pt").to(device)
    translated = model.generate(**batch, max_length=1024, num_beams=num_beams, num_return_sequences=num_return_sequences, temperature=temperature, do_sample = do_sample)
    tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
    for i in range(0,num_return_sequences,1):
        merged_strings = []
        for a in range(i,max_sentence_output,num_return_sequences):
            text = "".join(tgt_text[a])
            merged_strings.append("".join(tgt_text[a]))
        paraphrased_text = " ".join(merged_strings)
        BLUE_score,TER_score,ROUGE_score,METEOR_score,SACREBLUE_score = eval(predict_text= paraphrased_text, reference_text= input_string)
        dict_in_dict[i] = {
        "text": f"{paraphrased_text}",
        "BLUE_score": BLUE_score['bleu'],
        "TER_score": TER_score['score'],
        "ROUGE1_score": ROUGE_score['rouge1'],
        "ROUGEL_score": ROUGE_score['rougeL'],
        "METEOR_score": METEOR_score['meteor'],
        "SACREBLUE_score": SACREBLUE_score['score'],
    }
        
    return dict_in_dict

In [5]:

def eval(predict_text, reference_text):

    BLUE_score = bleu.compute(**{
        'predictions': [predict_text],
        'references': [reference_text]
    })
    TER_score = ter.compute(**{
        'predictions': [predict_text],
        'references': [reference_text]
    })
    ROUGE_score = rouge.compute(**{
        'predictions': [predict_text],
        'references': [reference_text]
    })
    METEOR_score = meteor.compute(**{
        'predictions': [predict_text],
        'references': [reference_text]
    })
    SACREBLUE_score = sacrebleu.compute(**{
        'predictions': [predict_text],
        'references': [reference_text]
    })

    return BLUE_score, TER_score, ROUGE_score, METEOR_score, SACREBLUE_score

In [9]:
input_string = input()
splitter = SentenceSplitter(language='en')
sentence_list = splitter.split(input_string)
output_dict = demo(sentence_list,10,10,1.5,True,input_string)

In [10]:
output_dict

{0: {'text': "Where did they get a random paragraph? They're writing a random paragraph, or do they get it somewhere? At that moment, he read the random paragraph and realized it was about random paragraphs and his world would never be the same.",
  'BLUE_score': 0.6133577802233244,
  'TER_score': 31.372549019607842,
  'ROUGE1_score': 0.8387096774193549,
  'ROUGEL_score': 0.8387096774193549,
  'METEOR_score': 0.7969403863748856,
  'SACREBLUE_score': 61.33577802233245},
 1: {'text': 'Where did they get a random paragraph? Or did they just write a random paragraph? At that moment, he read the random paragraph and realized it was about random paragraphs and his world would never be the same.',
  'BLUE_score': 0.5465181909576255,
  'TER_score': 37.254901960784316,
  'ROUGE1_score': 0.8089887640449439,
  'ROUGEL_score': 0.7865168539325842,
  'METEOR_score': 0.748126551219948,
  'SACREBLUE_score': 54.65181909576255},
 2: {'text': "Where did they get a random paragraph? They're writing a rand

In [1]:


import csv
import pandas as pd
import numpy as np
import torch
from comet_ml import Experiment
from tqdm.auto import tqdm
from datasets import load_metric

import tensorflow_hub as hub
import tensorflow as tf
import tensorflow_text as text  # Needed for loading universal-sentence-encoder-cmlm/multilingual-preprocess
import numpy as np

from transformers import PegasusForConditionalGeneration, PegasusTokenizer, get_cosine_schedule_with_warmup, get_constant_schedule_with_warmup
from transformers.models.bart.modeling_bart import shift_tokens_right
# from transformers import T5ForConditionalGeneration, T5Tokenizer, PegasusForConditionalGeneration, PegasusTokenizer
from sklearn.model_selection import train_test_split

import pickle

e:\THESIS\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [1]:
import pandas as pd
def read_dataset(path):
  dataframe = pd.read_parquet(path)
  return dataframe[['#1 String', '#2 String']]

,#1 String,#2 String,label
0,"Amrozi accused his brother, whom he called ""th...","Referring to him as only ""the witness"", Amrozi...",1
1,They had published an advertisement on the Int...,"On June 10, the ship's owners had published an...",1
2,Revenue in the first quarter of the year dropp...,With the scandal hanging over Stewart's compan...,1
3,The DVD-CCA then appealed to the state Supreme...,The DVD CCA appealed that decision to the U.S....,1
4,He said the foodservice pie business doesn't f...,"""The foodservice pie business does not fit our...",1
...,...,...,...
194326,Caffiers is a commune . It is found in the reg...,Caffiers is a commune in the Pas-de-Calais dep...,1
194327,549 people were living in Orange as of 2000 .,The population was 549 at the 2000 census .,1
194328,Orange is a town of Juneau County in the state...,"Orange is a town in Juneau County , Wisconsin ...",1
194329,Orainville is a commune . It is found in the r...,Orainville is a commune in the Aisne departmen...,1


In [29]:
dataset_train['#1 String'] = dataset_train['#1 String'].astype('str')
dataset_train['#2 String'] = dataset_train['#2 String'].astype('str')

In [30]:
dataset_train.dropna(inplace=True)

In [32]:
import random
random_ilocs = random.sample(list(dataset_train.index), 40000)


In [33]:
newdata = dataset_train.iloc[random_ilocs]

In [34]:
newdata.to_parquet('train_quality_dataset_40k.parquet')

In [35]:
dataaaaa = pd.read_parquet('../train_quality_dataset222.parquet')

In [36]:
from transformers import PegasusForConditionalGeneration, PegasusTokenizer 
from transformers.models.bart.modeling_bart import shift_tokens_right
model = PegasusForConditionalGeneration.from_pretrained('ThanhJamieAI/ParapharseV8_8E_4B')
tokenizer = PegasusTokenizer.from_pretrained('ThanhJamieAI/ParapharseV8_8E_4B')
dataset_train = dataaaaa
data_x = tokenizer(dataset_train['#1 String'].tolist(), padding='max_length',
                 max_length=dataset_train['#1 String'].map(lambda x: len(x)).max(), return_tensors='pt')

data_y = None

with tokenizer.as_target_tokenizer():
  data_y = tokenizer(dataset_train['#2 String'].tolist(), padding='max_length',
                          max_length=dataset_train['#2 String'].map(lambda x: len(x)).max(), return_tensors='np')
data_y = data_y['input_ids']
data_y[data_y == model.config.pad_token_id] = -100

data_x['labels'] = torch.tensor(data_y)

data_x['decoder_input_ids'] = shift_tokens_right(data_x['labels'], model.config.pad_token_id, model.config.decoder_start_token_id)

data_x

In [23]:
len(data_y)

50000